In [49]:
import xml.etree.ElementTree as ET
import re
from contextlib import suppress
import json

In [32]:
import nltk
from nltk.corpus import wordnet as ewn

def sc2ss(sensekey):
    '''Look up a synset given the information from SemCor'''
    ### Assuming it is the same WN version (e.g. 3.0)
    return ewn.lemma_from_key(sensekey).synset()
 
ss = sc2ss('live%2:42:06::') 
#print(ss)
print(ss.offset())
print (ss, ss.definition(), ss.lexname()) #'(%08d-%s)' % (ss.offset, ss.pos) )

2614387
Synset('live.v.02') lead a certain kind of life; live in a certain style verb.stative


In [62]:
corpus = {}
with open('./tagfiles/br-r05.xml') as f:
    for line in f:
        if '<wf' in line:
            root = ET.fromstring(line)
            if 'lemma' in root.attrib and 'lexsn' in root.attrib:
                word = root.attrib['lemma']
                lexsn = root.attrib['lexsn']
                
                pos = root.attrib['pos']
                try:
                    ss = sc2ss(word + '%' + lexsn)
                except:
                    pass
                
                temp = {ss : {'pos':pos, 'synset':ss, 'lemma':word}}
                
                
                corpus[ss] = temp 


In [63]:
corpus

{Synset('nothing.n.01'): {'pos': 'NN',
  'synset': Synset('nothing.n.01'),
  'lemma': 'nothing'},
 Synset('english.n.01'): {'pos': 'NN',
  'synset': Synset('english.n.01'),
  'lemma': 'english'},
 Synset('ridicule.v.01'): {'pos': 'VB',
  'synset': Synset('ridicule.v.01'),
  'lemma': 'ridicule'},
 Synset('equivocal.a.01'): {'pos': 'JJ',
  'synset': Synset('equivocal.a.01'),
  'lemma': 'ambiguous'},
 Synset('use.n.01'): {'pos': 'NN',
  'synset': Synset('use.n.01'),
  'lemma': 'use'},
 Synset('word.n.01'): {'pos': 'NN',
  'synset': Synset('word.n.01'),
  'lemma': 'word'},
 Synset('be.v.02'): {'pos': 'VB', 'synset': Synset('be.v.02'), 'lemma': 'be'},
 Synset('sentence.n.01'): {'pos': 'NN',
  'synset': Synset('sentence.n.01'),
  'lemma': 'sentence'},
 Synset('person.n.01'): {'pos': 'NNP',
  'synset': Synset('person.n.01'),
  'lemma': 'person'},
 Synset('state.v.01'): {'pos': 'VB',
  'synset': Synset('state.v.01'),
  'lemma': 'say'},
 Synset('intelligibly.r.01'): {'pos': 'RB',
  'synset': Sy

In [50]:
root.text

'in'

In [1]:
import os
import json
import re
import spacy
import time
from bson import json_util
from joblib import Parallel, delayed
import logging
import argparse
import xml.etree.ElementTree as ET
import nltk
from nltk.corpus import wordnet as ewn
from pathlib import Path
from functools import partial
#import thinc.extra.datasets
import plac
import spacy
from spacy.util import minibatch

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [45]:
def generate_path(root_path):
    path_list = []
    for root, dirs, files in os.walk(root_path):    
        file_list = [f for f in files if not f[0] == '.']
        
        for file in file_list:
            path_list.append(os.path.join(root, file))

    return path_list

In [47]:
generate_path('wordnet')

['wordnet/tagfiles/br-g07.xml',
 'wordnet/tagfiles/br-g13.xml',
 'wordnet/tagfiles/br-e05.xml',
 'wordnet/tagfiles/br-e11.xml',
 'wordnet/tagfiles/br-c17.xml',
 'wordnet/tagfiles/br-c03.xml',
 'wordnet/tagfiles/br-a29.xml',
 'wordnet/tagfiles/br-a28.xml',
 'wordnet/tagfiles/br-c16.xml',
 'wordnet/tagfiles/br-e10.xml',
 'wordnet/tagfiles/br-g06.xml',
 'wordnet/tagfiles/br-g10.xml',
 'wordnet/tagfiles/br-g04.xml',
 'wordnet/tagfiles/br-e12.xml',
 'wordnet/tagfiles/br-e06.xml',
 'wordnet/tagfiles/br-c14.xml',
 'wordnet/tagfiles/br-a16.xml',
 'wordnet/tagfiles/br-a17.xml',
 'wordnet/tagfiles/br-a03.xml',
 'wordnet/tagfiles/br-c15.xml',
 'wordnet/tagfiles/br-e07.xml',
 'wordnet/tagfiles/br-e13.xml',
 'wordnet/tagfiles/br-p08.xml',
 'wordnet/tagfiles/br-g05.xml',
 'wordnet/tagfiles/br-e17.xml',
 'wordnet/tagfiles/br-e03.xml',
 'wordnet/tagfiles/br-c05.xml',
 'wordnet/tagfiles/br-c11.xml',
 'wordnet/tagfiles/br-a07.xml',
 'wordnet/tagfiles/br-a06.xml',
 'wordnet/tagfiles/br-c10.xml',
 'wordne

In [62]:
semcor = {}
lc = 0
for file_path in generate_path('wordnet'):
    with open(file_path) as f:
        for line in f:
            semcor.update(json.loads(line))
            lc += 1


In [63]:
len(semcor)

25819

In [64]:
lc

224716

In [72]:
{semcor[s]['pos'] for s in semcor}

{'JJ', 'NN', 'NNP', 'RB', 'VB'}

In [77]:
semcor_noun = {s for s in semcor if (semcor[s]['pos'] == 'NN' or semcor[s]['pos'] == 'NNP')}

In [78]:
len(semcor_noun)

13125